In [1]:
from web3 import Web3, IPCProvider

In [2]:
w3 = Web3(IPCProvider('/home/sandipan/.ethereum/geth.ipc'))

In [3]:
w3.eth.getBlock('latest')

AttributeDict({'difficulty': 1022306478107570,
 'extraData': HexBytes('0x65746865726d696e652d757332'),
 'gasLimit': 6680831,
 'gasUsed': 6662976,
 'hash': HexBytes('0x01e66a59aa554541b806c9c59f9a8dd82a647679b1f14d726abd0cf018bc03aa'),
 'logsBloom': HexBytes('0x1000112400200138104080000181440000061000200320a4202001010108063010028a19002000000100429001103800404020001842608080042504220008004200020006001001e0104808304000000e408248202d000000810050000021802010c088960902000400008000001800822400004800010000262c1080041108a5a1020c800000004080000044011840001c9110008820d31004a402480400008006000002020008860100080404000020018881a0082600011081604802c010100000a280000022458c000044024e220204010000020c2008400008000021004180100000120010006000669000000801bd0800a42000084000000811000812'),
 'miner': '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8',
 'mixHash': HexBytes('0xdadc422814a92c4a1d2eeb9423df7b43b39d9c1b07b3cdea824690a49219e308'),
 'nonce': HexBytes('0xcf2238e002620bbc'),
 'number': 3981771,
 'parentHash'

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('transaction_till_100000.csv')

In [6]:
df.head()

,From,To,Value,Time,Block
0,ethereum,0,5.000000e+18,0,0
1,ethereum,05a56e2d52c817161883f50c441c3228cfe54d9f,5.000000e+18,1438269988,1
2,ethereum,0193d941b50d91be6567c7ee1c0fe7af498b4137,5.000000e+18,1438270128,10
3,ethereum,e6a7a1d47ff21b6321162aea7c6cb457d5476bca,5.000000e+18,1439799138,100000
4,ethereum,9beb4d346c6309a9ce4f27393c6322c8f03a3be6,5.000000e+18,1438336146,10099


In [17]:
add = df.From.unique()[1]


In [20]:
add = Web3.toChecksumAddress(add)

In [22]:
w3.eth.getBalance(add,100)

140000000000000000000

In [26]:
df_b = df[df['Block']==1000][['From','To']]
    

In [34]:
import numpy as np

In [35]:
np.concatenate(df_b.From.unique(),df_b.To.unique())

TypeError: only integer scalar arrays can be converted to a scalar index

In [38]:
address = np.append(df_b.From.unique(),df_b.To.unique())

In [41]:
blockNum = 100
for i in range(len(address)):
    if address[i]!='ethereum':
        account = Web3.toChecksumAddress(address[i])
        balance = w3.eth.getBalance(account,blockNum)
        print(address[i],balance)

bb7b8287f3f0a933474a79eae42cbca977791171 151250000000000000000


In [81]:
def fillBalanceTable(blockNum,fname):
    df_b = df[df['Block']==blockNum][['From','To']]
    address = np.append(df_b.From.unique(),df_b.To.unique())
    #with open(fname,'a') as ft:
    for i in range(len(address)):
        if address[i]!='ethereum' and address[i]!='0':
            account = Web3.toChecksumAddress(address[i])
            balance = w3.eth.getBalance(account,blockNum)
            ft = open(fname,'a')
            ft.write(account+','+str(balance)+','+str(blockNum))
            ft.write('\n')
            ft.close()

In [49]:
fillBalanceTable(10000)

bf71642d7cbae8faf1cfdc6c1c48fcb45b15ed22 147343750000000000000


In [47]:
df[df.Block==10000]

,From,To,Value,Time,Block
103,ethereum,bf71642d7cbae8faf1cfdc6c1c48fcb45b15ed22,5.000000e+18,1438334627,10000


In [74]:
fs = open('blockAccountBalance1.csv','w')
fs.write('Account,Balance,Block\n')
fs.close()

In [57]:
for i in range(1,1000):
    fillBalanceTable(i,'blockAccountBalance.csv')

In [62]:
df_1 = pd.read_csv('blockAccountBalance.csv')

In [63]:
df_1.head()

,Account,Balance,Block
0,0x05a56E2D52c817161883f50c441c3228CFe54d9f,5000000000000000000,1
1,0xdd2F1e6e498202e86D8f5442Af596580A4f03c2C,5000000000000000000,2
2,0x5088D623ba0fcf0131E0897a91734A4D83596AA0,5156250000000000000,3
3,0xc8ebccC5F5689Fa8659d83713341e5aD19349448,3750000000000000000,3
4,0xc8ebccC5F5689Fa8659d83713341e5aD19349448,8906250000000000000,4


In [64]:
df_1.Block.max()

5000

In [61]:
for i in range(1000,5001):
    fillBalanceTable(i,'blockAccountBalance.csv')

In [82]:
def createBalanceCSV(start,end,fname):
    for i in range(start,end+1):
        fillBalanceTable(i,fname)
        print('finished for block: ',i)

In [67]:
from multiprocessing import Process

In [77]:
jobs = []

In [83]:
jobs.append(Process(target=createBalanceCSV, args=(5305,7500,"blockAccountBalance.csv")))
jobs.append(Process(target=createBalanceCSV, args=(7501,10000,"blockAccountBalance.csv")))
jobs.append(Process(target=createBalanceCSV, args=(10001,12500,"blockAccountBalance.csv")))
jobs.append(Process(target=createBalanceCSV, args=(12501,15000,"blockAccountBalance.csv")))

In [84]:
for j in jobs:
    j.start()
    for j in jobs:
        j.join()

AssertionError: cannot start a process twice

In [80]:
df[df.Block==5305]

,From,To,Value,Time,Block
51641,ethereum,0,5.000000e+18,1438288819,5305


In [7]:
def balanceAccountBlock(blockNum, df): # retrieve balances of all accounts given a block number
    df_b = df[df['Block']<=blockNum][['From','To']]
    address = np.append(df_b.From.unique(),df_b.To.unique())
    with open('AccountBalanace'+str(blockNum)+'.csv','w') as ft:
        ft.write('Account,Balance,Block')
        ft.write('\n')
        for add in address:
            if address[i]!='ethereum' and address[i]!='0':
                account = Web3.toChecksumAddress(address[i])
                balance = w3.eth.getBalance(account,blockNum)
                ft.write(account+','+str(balance)+','+str(blockNum))
                ft.write('\n')
                ft.close() 
    